In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.layers import Input
from keras.models import Model

def create_vgg_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')


model model


In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator


def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=9):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 64
        steps_train = len(curr_y)//bat_size
        
        
        model = create_vgg_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=8)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.68599, saving model to best_m.h5
 - 8s - loss: 0.6984 - acc: 0.5395 - val_loss: 0.6860 - val_acc: 0.5274
Epoch 2/100
Epoch 00002: val_loss improved from 0.68599 to 0.68189, saving model to best_m.h5
 - 5s - loss: 0.6639 - acc: 0.5444 - val_loss: 0.6819 - val_acc: 0.5871
Epoch 3/100
Epoch 00003: val_loss improved from 0.68189 to 0.55200, saving model to best_m.h5
 - 5s - loss: 0.6433 - acc: 0.5695 - val_loss: 0.5520 - val_acc: 0.6965
Epoch 4/100
Epoch 00004: val_loss improved from 0.55200 to 0.50944, saving model to best_m.h5
 - 5s - loss: 0.5566 - acc: 0.7241 - val_loss: 0.5094 - val_acc: 0.8010
Epoch 5/100
Epoch 00005: val_loss improved from 0.50944 to 0.50756, saving model to best_m.h5
 - 5s - loss: 0.5492 - acc: 0.7254 - val_loss: 0.5076 - val_acc: 0.7711
Epoch 6/100
Epoch 00006: val_loss improved from 0.50756 to 0.47882, saving model to best_m.h5
 - 5s - loss: 0.4854 - acc: 0.7899 - val_loss: 0.4788 - val_acc: 0.8010
Epoch 7/

Epoch 61/100
Epoch 00061: val_loss did not improve
 - 4s - loss: 0.1750 - acc: 0.9201 - val_loss: 0.2581 - val_acc: 0.9104
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 4s - loss: 0.1746 - acc: 0.9186 - val_loss: 0.2979 - val_acc: 0.8955
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 4s - loss: 0.1773 - acc: 0.9262 - val_loss: 0.2969 - val_acc: 0.8806
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 4s - loss: 0.1597 - acc: 0.9403 - val_loss: 0.2679 - val_acc: 0.9055
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 4s - loss: 0.1886 - acc: 0.9254 - val_loss: 0.2746 - val_acc: 0.8905
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 4s - loss: 0.1715 - acc: 0.9274 - val_loss: 0.2662 - val_acc: 0.9104
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 4s - loss: 0.1657 - acc: 0.9314 - val_loss: 0.2735 - val_acc: 0.9154
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 4s - loss: 0.1790 - acc: 0.9232 - val_loss: 0.2639 - val_acc: 0.9104
Epoch 69/100
Epo

Epoch 24/100
Epoch 00024: val_loss did not improve
 - 4s - loss: 0.2080 - acc: 0.9067 - val_loss: 0.4280 - val_acc: 0.8209
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 4s - loss: 0.2608 - acc: 0.8940 - val_loss: 0.2812 - val_acc: 0.8607
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 4s - loss: 0.2261 - acc: 0.9023 - val_loss: 0.2965 - val_acc: 0.8507
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 4s - loss: 0.2527 - acc: 0.8958 - val_loss: 0.3199 - val_acc: 0.8607
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 4s - loss: 0.2491 - acc: 0.8915 - val_loss: 0.3123 - val_acc: 0.8507
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 4s - loss: 0.2350 - acc: 0.9030 - val_loss: 0.2766 - val_acc: 0.8756
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 4s - loss: 0.1996 - acc: 0.9155 - val_loss: 0.2712 - val_acc: 0.8856
Epoch 31/100
Epoch 00031: val_loss did not improve
 - 4s - loss: 0.1861 - acc: 0.9269 - val_loss: 0.2832 - val_acc: 0.8706
Epoch 32/100
Epo

Epoch 90/100
Epoch 00090: val_loss did not improve
 - 4s - loss: 0.1701 - acc: 0.9336 - val_loss: 0.2796 - val_acc: 0.8706
Epoch 91/100
Epoch 00091: val_loss did not improve
 - 4s - loss: 0.1427 - acc: 0.9389 - val_loss: 0.2939 - val_acc: 0.8706
Epoch 92/100
Epoch 00092: val_loss did not improve
 - 4s - loss: 0.1450 - acc: 0.9350 - val_loss: 0.2902 - val_acc: 0.8806
Epoch 93/100
Epoch 00093: val_loss did not improve
 - 4s - loss: 0.1370 - acc: 0.9410 - val_loss: 0.3121 - val_acc: 0.8607
Epoch 94/100
Epoch 00094: val_loss did not improve
 - 4s - loss: 0.1330 - acc: 0.9479 - val_loss: 0.2804 - val_acc: 0.8756
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 4s - loss: 0.1637 - acc: 0.9298 - val_loss: 0.2641 - val_acc: 0.8955
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 4s - loss: 0.1609 - acc: 0.9344 - val_loss: 0.2831 - val_acc: 0.8756
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 4s - loss: 0.1326 - acc: 0.9431 - val_loss: 0.3063 - val_acc: 0.8657
Epoch 98/100
Epo

Epoch 49/100
Epoch 00049: val_loss did not improve
 - 4s - loss: 0.2247 - acc: 0.9036 - val_loss: 0.1737 - val_acc: 0.9254
Epoch 50/100
Epoch 00050: val_loss did not improve
 - 4s - loss: 0.2149 - acc: 0.9042 - val_loss: 0.1636 - val_acc: 0.9154
Epoch 51/100
Epoch 00051: val_loss did not improve
 - 5s - loss: 0.1954 - acc: 0.9159 - val_loss: 0.1582 - val_acc: 0.9353
Epoch 52/100
Epoch 00052: val_loss did not improve
 - 4s - loss: 0.2249 - acc: 0.9036 - val_loss: 0.1588 - val_acc: 0.9303
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 4s - loss: 0.2123 - acc: 0.8985 - val_loss: 0.1793 - val_acc: 0.9204
Epoch 54/100
Epoch 00054: val_loss did not improve
 - 4s - loss: 0.1985 - acc: 0.9207 - val_loss: 0.1765 - val_acc: 0.9204
Epoch 55/100
Epoch 00055: val_loss did not improve
 - 4s - loss: 0.2078 - acc: 0.9122 - val_loss: 0.1594 - val_acc: 0.9254
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 4s - loss: 0.1948 - acc: 0.9145 - val_loss: 0.1594 - val_acc: 0.9353
Epoch 57/100
Epo

...

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.64785, saving model to best_m.h5
 - 5s - loss: 0.6756 - acc: 0.5430 - val_loss: 0.6479 - val_acc: 0.4850
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 4s - loss: 0.6601 - acc: 0.6136 - val_loss: 0.6728 - val_acc: 0.5350
Epoch 3/100
Epoch 00003: val_loss improved from 0.64785 to 0.50147, saving model to best_m.h5
 - 5s - loss: 0.5670 - acc: 0.6756 - val_loss: 0.5015 - val_acc: 0.8050
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 4s - loss: 0.6144 - acc: 0.6127 - val_loss: 0.5749 - val_acc: 0.8000
Epoch 5/100
Epoch 00005: val_loss improved from 0.50147 to 0.34569, saving model to best_m.h5
 - 5s - loss: 0.4991 - acc: 0.7543 - val_loss: 0.3457 - val_acc: 0.8900
Epoch 6/100
Epoch 00006: val_loss improved from 0.34569 to 0.33062, saving model to best_m.h5
 - 5s - loss: 0.4380 - acc: 0.8051 - val_loss: 0.3306 - val_acc: 0.8800
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 4s - loss: 0.4466 - acc: 0.8140 - val_loss:

Epoch 63/100
Epoch 00063: val_loss did not improve
 - 4s - loss: 0.1819 - acc: 0.9269 - val_loss: 0.1460 - val_acc: 0.9200
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 4s - loss: 0.1891 - acc: 0.9194 - val_loss: 0.1590 - val_acc: 0.9100
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 4s - loss: 0.1734 - acc: 0.9298 - val_loss: 0.1537 - val_acc: 0.9100
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 4s - loss: 0.1863 - acc: 0.9208 - val_loss: 0.1512 - val_acc: 0.9200
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 4s - loss: 0.1867 - acc: 0.9171 - val_loss: 0.1763 - val_acc: 0.9050
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 4s - loss: 0.1792 - acc: 0.9270 - val_loss: 0.1602 - val_acc: 0.9150
Epoch 69/100
Epoch 00069: val_loss improved from 0.13237 to 0.12596, saving model to best_m.h5
 - 5s - loss: 0.1696 - acc: 0.9217 - val_loss: 0.1260 - val_acc: 0.9450
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 4s - loss: 0.1766 - acc: 0.9329 - val_los

Epoch 00024: val_loss did not improve
 - 4s - loss: 0.2791 - acc: 0.8762 - val_loss: 0.2943 - val_acc: 0.8850
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 4s - loss: 0.3103 - acc: 0.8683 - val_loss: 0.2373 - val_acc: 0.9000
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 4s - loss: 0.2802 - acc: 0.8899 - val_loss: 0.2668 - val_acc: 0.8550
Epoch 27/100
Epoch 00027: val_loss improved from 0.20249 to 0.20168, saving model to best_m.h5
 - 5s - loss: 0.2818 - acc: 0.8713 - val_loss: 0.2017 - val_acc: 0.9300
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 4s - loss: 0.2478 - acc: 0.8971 - val_loss: 0.2054 - val_acc: 0.8950
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 4s - loss: 0.2473 - acc: 0.8926 - val_loss: 0.2050 - val_acc: 0.9450
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 4s - loss: 0.2486 - acc: 0.8978 - val_loss: 0.2088 - val_acc: 0.9350
Epoch 31/100
Epoch 00031: val_loss improved from 0.20168 to 0.19517, saving model to best_m.h5
 - 5s - loss:

Epoch 87/100
Epoch 00087: val_loss did not improve
 - 4s - loss: 0.1840 - acc: 0.9201 - val_loss: 0.1652 - val_acc: 0.9450
Epoch 88/100
Epoch 00088: val_loss did not improve
 - 4s - loss: 0.1653 - acc: 0.9328 - val_loss: 0.1798 - val_acc: 0.9200
Epoch 89/100
Epoch 00089: val_loss did not improve
 - 4s - loss: 0.1954 - acc: 0.9163 - val_loss: 0.2013 - val_acc: 0.9250
Epoch 90/100
Epoch 00090: val_loss did not improve
 - 4s - loss: 0.1718 - acc: 0.9275 - val_loss: 0.1811 - val_acc: 0.9350
Epoch 91/100
Epoch 00091: val_loss did not improve
 - 4s - loss: 0.1920 - acc: 0.9127 - val_loss: 0.2293 - val_acc: 0.9000
Epoch 92/100
Epoch 00092: val_loss did not improve
 - 4s - loss: 0.1849 - acc: 0.9163 - val_loss: 0.1557 - val_acc: 0.9450
Epoch 93/100
Epoch 00093: val_loss did not improve
 - 4s - loss: 0.1677 - acc: 0.9256 - val_loss: 0.1606 - val_acc: 0.9300
Epoch 94/100
Epoch 00094: val_loss did not improve
 - 4s - loss: 0.1666 - acc: 0.9283 - val_loss: 0.1784 - val_acc: 0.9300
Epoch 95/100
Epo

Epoch 00047: val_loss did not improve
 - 4s - loss: 0.2191 - acc: 0.9066 - val_loss: 0.1890 - val_acc: 0.9150
Epoch 48/100
Epoch 00048: val_loss did not improve
 - 4s - loss: 0.2079 - acc: 0.9185 - val_loss: 0.1864 - val_acc: 0.9150
Epoch 49/100
Epoch 00049: val_loss did not improve
 - 4s - loss: 0.2251 - acc: 0.9097 - val_loss: 0.1854 - val_acc: 0.9200
Epoch 50/100
Epoch 00050: val_loss did not improve
 - 5s - loss: 0.2097 - acc: 0.9152 - val_loss: 0.1823 - val_acc: 0.9150
Epoch 51/100
Epoch 00051: val_loss did not improve
 - 4s - loss: 0.2098 - acc: 0.9064 - val_loss: 0.1819 - val_acc: 0.9150
Epoch 52/100
Epoch 00052: val_loss did not improve
 - 5s - loss: 0.2112 - acc: 0.9122 - val_loss: 0.1833 - val_acc: 0.9200
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 5s - loss: 0.2136 - acc: 0.9098 - val_loss: 0.1824 - val_acc: 0.9150
Epoch 54/100
Epoch 00054: val_loss improved from 0.18130 to 0.18004, saving model to best_m.h5
 - 5s - loss: 0.1999 - acc: 0.9141 - val_loss: 0.1800 - v

In [6]:
import pickle
with open('../features/vgg_aug1_fold8_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/vgg_aug1_fold8_sub.csv', index=False)

# pre 2370
# new 2088

0.197812022234
         id  is_iceberg
0  5941774d    0.039438
1  4023181e    0.810475
2  b20200e4    0.177227
3  e7f018bb    0.997459
4  4371c8c3    0.025433


In [7]:
from keras.callbacks import ModelCheckpoint
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten
from keras import layers
from keras.models import Model

def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):

    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    bn_axis = 3
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x

def create_incept_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # bn
    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
    x = conv2d_bn(x, 32, 3, 3, padding='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv2d_bn(x, 64, 1, 1, padding='valid')
    x = conv2d_bn(x, 64, 3, 3, padding='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # mixed 0, 1, 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')
    
    # mixed 1: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')
    
    # mixed 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')

    x = Conv2D(64, 3, strides=2, padding='same',activation='relu')(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')

model model


In [8]:
def kfold_train(fold_cnt=3,rnd=9):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_incept_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=8)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.72429, saving model to best_m.h5
 - 10s - loss: 0.4941 - acc: 0.7645 - val_loss: 0.7243 - val_acc: 0.4975
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 4s - loss: 0.4253 - acc: 0.7935 - val_loss: 1.0352 - val_acc: 0.5423
Epoch 3/100
Epoch 00003: val_loss improved from 0.72429 to 0.64460, saving model to best_m.h5
 - 4s - loss: 0.3721 - acc: 0.8215 - val_loss: 0.6446 - val_acc: 0.7463
Epoch 4/100
Epoch 00004: val_loss improved from 0.64460 to 0.46922, saving model to best_m.h5
 - 4s - loss: 0.3373 - acc: 0.8502 - val_loss: 0.4692 - val_acc: 0.7960
Epoch 5/100
Epoch 00005: val_loss did not improve
 - 4s - loss: 0.3735 - acc: 0.8459 - val_loss: 1.8300 - val_acc: 0.5970
Epoch 6/100
Epoch 00006: val_loss improved from 0.46922 to 0.28656, saving model to best_m.h5
 - 4s - loss: 0.3297 - acc: 0.8615 - val_loss: 0.2866 - val_acc: 0.8856
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 4s - loss: 0.3129 - acc: 0.8578 - val_loss

Epoch 65/100
Epoch 00065: val_loss did not improve
 - 4s - loss: 0.1597 - acc: 0.9347 - val_loss: 0.3515 - val_acc: 0.8806
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 4s - loss: 0.1440 - acc: 0.9422 - val_loss: 0.3962 - val_acc: 0.8856
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 4s - loss: 0.1441 - acc: 0.9411 - val_loss: 0.3906 - val_acc: 0.8955
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 4s - loss: 0.1417 - acc: 0.9368 - val_loss: 0.4230 - val_acc: 0.8905
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 4s - loss: 0.1441 - acc: 0.9440 - val_loss: 0.4318 - val_acc: 0.8955
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 4s - loss: 0.1277 - acc: 0.9469 - val_loss: 0.3966 - val_acc: 0.8955
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 4s - loss: 0.1593 - acc: 0.9307 - val_loss: 0.4052 - val_acc: 0.8905
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 4s - loss: 0.1369 - acc: 0.9372 - val_loss: 0.3775 - val_acc: 0.8955
Epoch 73/100
Epo

Epoch 28/100
Epoch 00028: val_loss did not improve
 - 4s - loss: 0.2278 - acc: 0.9110 - val_loss: 0.2989 - val_acc: 0.8657
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 4s - loss: 0.2374 - acc: 0.8955 - val_loss: 0.3201 - val_acc: 0.8458
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 4s - loss: 0.2298 - acc: 0.9027 - val_loss: 0.3178 - val_acc: 0.8408
Epoch 31/100
Epoch 00031: val_loss did not improve
 - 4s - loss: 0.2001 - acc: 0.9186 - val_loss: 0.2762 - val_acc: 0.8706
Epoch 32/100
Epoch 00032: val_loss did not improve
 - 4s - loss: 0.2212 - acc: 0.9143 - val_loss: 0.2691 - val_acc: 0.8756
Epoch 33/100
Epoch 00033: val_loss improved from 0.26594 to 0.24762, saving model to best_m.h5
 - 4s - loss: 0.1911 - acc: 0.9221 - val_loss: 0.2476 - val_acc: 0.8856
Epoch 34/100
Epoch 00034: val_loss did not improve
 - 4s - loss: 0.1937 - acc: 0.9207 - val_loss: 0.2527 - val_acc: 0.8806
Epoch 35/100
Epoch 00035: val_loss did not improve
 - 4s - loss: 0.1789 - acc: 0.9300 - val_los

Epoch 93/100
Epoch 00093: val_loss did not improve
 - 4s - loss: 0.1213 - acc: 0.9519 - val_loss: 0.2790 - val_acc: 0.8905
Epoch 94/100
Epoch 00094: val_loss did not improve
 - 4s - loss: 0.1370 - acc: 0.9461 - val_loss: 0.2713 - val_acc: 0.8905
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 4s - loss: 0.1044 - acc: 0.9508 - val_loss: 0.2972 - val_acc: 0.8756
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 4s - loss: 0.1244 - acc: 0.9515 - val_loss: 0.2519 - val_acc: 0.8905
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 4s - loss: 0.1269 - acc: 0.9526 - val_loss: 0.2839 - val_acc: 0.8806
Epoch 98/100
Epoch 00098: val_loss did not improve
 - 4s - loss: 0.1182 - acc: 0.9497 - val_loss: 0.2663 - val_acc: 0.8856
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 4s - loss: 0.1262 - acc: 0.9504 - val_loss: 0.3120 - val_acc: 0.8706
Epoch 100/100
Epoch 00100: val_loss did not improve
 - 4s - loss: 0.1312 - acc: 0.9501 - val_loss: 0.2572 - val_acc: 0.8756
Epoch 1/100
Epo

Epoch 56/100
Epoch 00056: val_loss did not improve
 - 4s - loss: 0.1624 - acc: 0.9271 - val_loss: 0.1565 - val_acc: 0.9303
Epoch 57/100
Epoch 00057: val_loss did not improve
 - 4s - loss: 0.1587 - acc: 0.9340 - val_loss: 0.1691 - val_acc: 0.9303
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 4s - loss: 0.1741 - acc: 0.9346 - val_loss: 0.1910 - val_acc: 0.9204
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 4s - loss: 0.1645 - acc: 0.9332 - val_loss: 0.1693 - val_acc: 0.9254
Epoch 60/100
Epoch 00060: val_loss did not improve
 - 4s - loss: 0.1644 - acc: 0.9411 - val_loss: 0.1555 - val_acc: 0.9303
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 4s - loss: 0.1726 - acc: 0.9325 - val_loss: 0.1748 - val_acc: 0.9453
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 4s - loss: 0.1607 - acc: 0.9379 - val_loss: 0.1948 - val_acc: 0.9303
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 4s - loss: 0.1334 - acc: 0.9454 - val_loss: 0.1772 - val_acc: 0.9303
Epoch 64/100
Epo

...

Epoch 00016: val_loss did not improve
 - 4s - loss: 0.2657 - acc: 0.8817 - val_loss: 0.2133 - val_acc: 0.9150
Epoch 17/100
Epoch 00017: val_loss did not improve
 - 4s - loss: 0.2454 - acc: 0.8942 - val_loss: 0.3469 - val_acc: 0.8250
Epoch 18/100
Epoch 00018: val_loss did not improve
 - 4s - loss: 0.2875 - acc: 0.8846 - val_loss: 0.2689 - val_acc: 0.8800
Epoch 19/100
Epoch 00019: val_loss did not improve
 - 4s - loss: 0.2530 - acc: 0.9030 - val_loss: 0.1911 - val_acc: 0.9300
Epoch 20/100
Epoch 00020: val_loss improved from 0.19058 to 0.15766, saving model to best_m.h5
 - 4s - loss: 0.2474 - acc: 0.9011 - val_loss: 0.1577 - val_acc: 0.9400
Epoch 21/100
Epoch 00021: val_loss improved from 0.15766 to 0.15696, saving model to best_m.h5
 - 4s - loss: 0.2633 - acc: 0.9040 - val_loss: 0.1570 - val_acc: 0.9200
Epoch 22/100
Epoch 00022: val_loss did not improve
 - 4s - loss: 0.2425 - acc: 0.8985 - val_loss: 0.1668 - val_acc: 0.9400
Epoch 23/100
Epoch 00023: val_loss did not improve
 - 4s - loss:

Epoch 82/100
Epoch 00082: val_loss did not improve
 - 4s - loss: 0.1361 - acc: 0.9492 - val_loss: 0.2645 - val_acc: 0.9100
Epoch 83/100
Epoch 00083: val_loss did not improve
 - 4s - loss: 0.1450 - acc: 0.9425 - val_loss: 0.2508 - val_acc: 0.8950
Epoch 84/100
Epoch 00084: val_loss did not improve
 - 4s - loss: 0.1412 - acc: 0.9447 - val_loss: 0.2119 - val_acc: 0.9250
Epoch 85/100
Epoch 00085: val_loss did not improve
 - 4s - loss: 0.1266 - acc: 0.9423 - val_loss: 0.2156 - val_acc: 0.8900
Epoch 86/100
Epoch 00086: val_loss did not improve
 - 4s - loss: 0.1213 - acc: 0.9476 - val_loss: 0.2785 - val_acc: 0.8800
Epoch 87/100
Epoch 00087: val_loss did not improve
 - 4s - loss: 0.1324 - acc: 0.9428 - val_loss: 0.2297 - val_acc: 0.9000
Epoch 88/100
Epoch 00088: val_loss did not improve
 - 4s - loss: 0.1237 - acc: 0.9452 - val_loss: 0.2263 - val_acc: 0.9150
Epoch 89/100
Epoch 00089: val_loss did not improve
 - 4s - loss: 0.1340 - acc: 0.9526 - val_loss: 0.2138 - val_acc: 0.9050
Epoch 90/100
Epo

Epoch 47/100
Epoch 00047: val_loss did not improve
 - 4s - loss: 0.1845 - acc: 0.9231 - val_loss: 0.2044 - val_acc: 0.9200
Epoch 48/100
Epoch 00048: val_loss did not improve
 - 4s - loss: 0.1633 - acc: 0.9368 - val_loss: 0.2130 - val_acc: 0.9150
Epoch 49/100
Epoch 00049: val_loss did not improve
 - 4s - loss: 0.1889 - acc: 0.9294 - val_loss: 0.2308 - val_acc: 0.9050
Epoch 50/100
Epoch 00050: val_loss did not improve
 - 4s - loss: 0.1668 - acc: 0.9291 - val_loss: 0.2372 - val_acc: 0.8950
Epoch 51/100
Epoch 00051: val_loss did not improve
 - 4s - loss: 0.1694 - acc: 0.9272 - val_loss: 0.2709 - val_acc: 0.8800
Epoch 52/100
Epoch 00052: val_loss did not improve
 - 4s - loss: 0.1821 - acc: 0.9282 - val_loss: 0.2543 - val_acc: 0.9000
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 4s - loss: 0.1684 - acc: 0.9236 - val_loss: 0.2012 - val_acc: 0.9100
Epoch 54/100
Epoch 00054: val_loss did not improve
 - 4s - loss: 0.1746 - acc: 0.9286 - val_loss: 0.1923 - val_acc: 0.9250
Epoch 55/100
Epo

 - 4s - loss: 0.3181 - acc: 0.8604 - val_loss: 0.1999 - val_acc: 0.9100
Epoch 12/100
Epoch 00012: val_loss improved from 0.19993 to 0.19642, saving model to best_m.h5
 - 4s - loss: 0.2796 - acc: 0.8800 - val_loss: 0.1964 - val_acc: 0.9100
Epoch 13/100
Epoch 00013: val_loss did not improve
 - 4s - loss: 0.2637 - acc: 0.8846 - val_loss: 0.1974 - val_acc: 0.9050
Epoch 14/100
Epoch 00014: val_loss improved from 0.19642 to 0.19250, saving model to best_m.h5
 - 4s - loss: 0.2725 - acc: 0.8944 - val_loss: 0.1925 - val_acc: 0.9050
Epoch 15/100
Epoch 00015: val_loss did not improve
 - 4s - loss: 0.2467 - acc: 0.8970 - val_loss: 0.3245 - val_acc: 0.8550
Epoch 16/100
Epoch 00016: val_loss did not improve
 - 4s - loss: 0.2632 - acc: 0.8772 - val_loss: 0.2257 - val_acc: 0.8850
Epoch 17/100
Epoch 00017: val_loss did not improve
 - 4s - loss: 0.2714 - acc: 0.8836 - val_loss: 0.2023 - val_acc: 0.9050
Epoch 18/100
Epoch 00018: val_loss did not improve
 - 4s - loss: 0.2457 - acc: 0.8958 - val_loss: 0.25

Epoch 75/100
Epoch 00075: val_loss did not improve
 - 4s - loss: 0.1588 - acc: 0.9389 - val_loss: 0.2191 - val_acc: 0.9000
Epoch 76/100
Epoch 00076: val_loss did not improve
 - 4s - loss: 0.1543 - acc: 0.9380 - val_loss: 0.1850 - val_acc: 0.9400
Epoch 77/100
Epoch 00077: val_loss did not improve
 - 4s - loss: 0.1649 - acc: 0.9380 - val_loss: 0.1965 - val_acc: 0.9050
Epoch 78/100
Epoch 00078: val_loss did not improve
 - 4s - loss: 0.1499 - acc: 0.9337 - val_loss: 0.1795 - val_acc: 0.9000
Epoch 79/100
Epoch 00079: val_loss did not improve
 - 4s - loss: 0.1504 - acc: 0.9437 - val_loss: 0.1798 - val_acc: 0.9250
Epoch 80/100
Epoch 00080: val_loss did not improve
 - 4s - loss: 0.1381 - acc: 0.9440 - val_loss: 0.1880 - val_acc: 0.9100
Epoch 81/100
Epoch 00081: val_loss did not improve
 - 4s - loss: 0.1617 - acc: 0.9315 - val_loss: 0.2184 - val_acc: 0.8900
Epoch 82/100
Epoch 00082: val_loss did not improve
 - 4s - loss: 0.1418 - acc: 0.9377 - val_loss: 0.2240 - val_acc: 0.9000
Epoch 83/100
Epo

In [9]:
with open('../features/incept_aug1_fold8_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/incept_aug1_fold8_sub.csv', index=False)

# pre 2256
# new 16 batch 1953
# new 64 batch 2063

0.196251751859
         id  is_iceberg
0  5941774d    0.117416
1  4023181e    0.888277
2  b20200e4    0.137550
3  e7f018bb    0.998634
4  4371c8c3    0.369225


In [10]:
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters

    bn_axis = 3
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_resnet_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    bn_axis = 3
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(img_input)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 96], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 96], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 96], stage=2, block='c')

    x = conv_block(x, 3, [96, 96, 128], stage=3, block='a')
    x = identity_block(x, 3, [96, 96, 128], stage=3, block='b')
    x = identity_block(x, 3, [96, 96, 128], stage=3, block='c')
    x = identity_block(x, 3, [96, 96, 128], stage=3, block='d')
    
    x = Conv2D(128, 3, strides=2, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')


model model


In [11]:
def kfold_train(fold_cnt=3,rnd=9):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_resnet_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=8)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.72599, saving model to best_m.h5
 - 43s - loss: 0.9175 - acc: 0.5827 - val_loss: 0.7260 - val_acc: 0.4726
Epoch 2/100
Epoch 00002: val_loss improved from 0.72599 to 0.69228, saving model to best_m.h5
 - 5s - loss: 0.5267 - acc: 0.7590 - val_loss: 0.6923 - val_acc: 0.5323
Epoch 3/100
Epoch 00003: val_loss improved from 0.69228 to 0.63009, saving model to best_m.h5
 - 5s - loss: 0.4323 - acc: 0.7996 - val_loss: 0.6301 - val_acc: 0.7313
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 5s - loss: 0.4046 - acc: 0.8086 - val_loss: 0.6479 - val_acc: 0.7662
Epoch 5/100
Epoch 00005: val_loss improved from 0.63009 to 0.59015, saving model to best_m.h5
 - 5s - loss: 0.3897 - acc: 0.8217 - val_loss: 0.5902 - val_acc: 0.7861
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 5s - loss: 0.3904 - acc: 0.8083 - val_loss: 0.5991 - val_acc: 0.7562
Epoch 7/100
Epoch 00007: val_loss improved from 0.59015 to 0.40865, saving model to best_m.h5
 

Epoch 65/100
Epoch 00065: val_loss did not improve
 - 5s - loss: 0.1966 - acc: 0.9095 - val_loss: 0.2753 - val_acc: 0.8806
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 5s - loss: 0.2171 - acc: 0.9073 - val_loss: 0.2648 - val_acc: 0.8856
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 5s - loss: 0.1886 - acc: 0.9092 - val_loss: 0.2939 - val_acc: 0.8806
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 5s - loss: 0.1761 - acc: 0.9239 - val_loss: 0.3018 - val_acc: 0.8806
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 5s - loss: 0.2202 - acc: 0.9030 - val_loss: 0.2902 - val_acc: 0.8806
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 5s - loss: 0.1904 - acc: 0.9124 - val_loss: 0.2983 - val_acc: 0.8806
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 5s - loss: 0.2004 - acc: 0.9156 - val_loss: 0.3050 - val_acc: 0.8657
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 5s - loss: 0.2016 - acc: 0.9224 - val_loss: 0.2901 - val_acc: 0.8806
Epoch 73/100
Epo

Epoch 28/100
Epoch 00028: val_loss did not improve
 - 5s - loss: 0.2625 - acc: 0.8915 - val_loss: 0.3067 - val_acc: 0.8706
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 5s - loss: 0.2597 - acc: 0.8866 - val_loss: 0.3167 - val_acc: 0.8607
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 5s - loss: 0.2564 - acc: 0.8930 - val_loss: 0.3204 - val_acc: 0.8358
Epoch 31/100
Epoch 00031: val_loss did not improve
 - 5s - loss: 0.2411 - acc: 0.9010 - val_loss: 0.2829 - val_acc: 0.8856
Epoch 32/100
Epoch 00032: val_loss improved from 0.27876 to 0.27096, saving model to best_m.h5
 - 5s - loss: 0.2137 - acc: 0.9020 - val_loss: 0.2710 - val_acc: 0.8806
Epoch 33/100
Epoch 00033: val_loss did not improve
 - 5s - loss: 0.1953 - acc: 0.9178 - val_loss: 0.2733 - val_acc: 0.8806
Epoch 34/100
Epoch 00034: val_loss did not improve
 - 5s - loss: 0.2504 - acc: 0.8951 - val_loss: 0.2865 - val_acc: 0.8806
Epoch 35/100
Epoch 00035: val_loss did not improve
 - 5s - loss: 0.2291 - acc: 0.9099 - val_los

Epoch 93/100
Epoch 00093: val_loss did not improve
 - 5s - loss: 0.1730 - acc: 0.9325 - val_loss: 0.2979 - val_acc: 0.8657
Epoch 94/100
Epoch 00094: val_loss did not improve
 - 5s - loss: 0.1844 - acc: 0.9282 - val_loss: 0.2671 - val_acc: 0.8756
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 5s - loss: 0.1559 - acc: 0.9357 - val_loss: 0.4244 - val_acc: 0.7910
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 5s - loss: 0.1776 - acc: 0.9235 - val_loss: 0.2924 - val_acc: 0.8905
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 5s - loss: 0.1653 - acc: 0.9310 - val_loss: 0.3278 - val_acc: 0.8607
Epoch 98/100
Epoch 00098: val_loss did not improve
 - 5s - loss: 0.1457 - acc: 0.9375 - val_loss: 0.3301 - val_acc: 0.8507
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 5s - loss: 0.1839 - acc: 0.9268 - val_loss: 0.3249 - val_acc: 0.8905
Epoch 100/100
Epoch 00100: val_loss did not improve
 - 5s - loss: 0.1533 - acc: 0.9350 - val_loss: 0.2838 - val_acc: 0.8507
Epoch 1/100
Epo

Epoch 58/100
Epoch 00058: val_loss did not improve
 - 5s - loss: 0.2153 - acc: 0.9020 - val_loss: 0.1582 - val_acc: 0.9453
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 5s - loss: 0.2332 - acc: 0.8987 - val_loss: 0.1645 - val_acc: 0.9403
Epoch 60/100
Epoch 00060: val_loss did not improve
 - 5s - loss: 0.2176 - acc: 0.9013 - val_loss: 0.1798 - val_acc: 0.9403
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 5s - loss: 0.2172 - acc: 0.9095 - val_loss: 0.1633 - val_acc: 0.9254
Epoch 62/100
Epoch 00062: val_loss improved from 0.15437 to 0.15391, saving model to best_m.h5
 - 5s - loss: 0.2071 - acc: 0.9045 - val_loss: 0.1539 - val_acc: 0.9453
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 5s - loss: 0.2102 - acc: 0.9142 - val_loss: 0.1545 - val_acc: 0.9453
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 5s - loss: 0.2150 - acc: 0.9131 - val_loss: 0.1552 - val_acc: 0.9453
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 5s - loss: 0.2572 - acc: 0.8909 - val_los

...

Epoch 13/100
Epoch 00013: val_loss did not improve
 - 5s - loss: 0.3312 - acc: 0.8441 - val_loss: 0.3213 - val_acc: 0.8200
Epoch 14/100
Epoch 00014: val_loss improved from 0.25191 to 0.22507, saving model to best_m.h5
 - 6s - loss: 0.3317 - acc: 0.8364 - val_loss: 0.2251 - val_acc: 0.9150
Epoch 15/100
Epoch 00015: val_loss improved from 0.22507 to 0.20506, saving model to best_m.h5
 - 6s - loss: 0.3162 - acc: 0.8625 - val_loss: 0.2051 - val_acc: 0.9050
Epoch 16/100
Epoch 00016: val_loss did not improve
 - 5s - loss: 0.2937 - acc: 0.8661 - val_loss: 0.2170 - val_acc: 0.9100
Epoch 17/100
Epoch 00017: val_loss did not improve
 - 5s - loss: 0.3060 - acc: 0.8628 - val_loss: 0.2090 - val_acc: 0.9100
Epoch 18/100
Epoch 00018: val_loss did not improve
 - 5s - loss: 0.3321 - acc: 0.8558 - val_loss: 0.2387 - val_acc: 0.8600
Epoch 19/100
Epoch 00019: val_loss improved from 0.20506 to 0.20209, saving model to best_m.h5
 - 6s - loss: 0.3311 - acc: 0.8520 - val_loss: 0.2021 - val_acc: 0.8950
Epoch 2

Epoch 77/100
Epoch 00077: val_loss did not improve
 - 5s - loss: 0.2279 - acc: 0.9071 - val_loss: 0.2473 - val_acc: 0.9050
Epoch 78/100
Epoch 00078: val_loss did not improve
 - 5s - loss: 0.1919 - acc: 0.9198 - val_loss: 0.1784 - val_acc: 0.9150
Epoch 79/100
Epoch 00079: val_loss did not improve
 - 5s - loss: 0.2090 - acc: 0.9114 - val_loss: 0.1791 - val_acc: 0.9350
Epoch 80/100
Epoch 00080: val_loss did not improve
 - 5s - loss: 0.1968 - acc: 0.9159 - val_loss: 0.1756 - val_acc: 0.9150
Epoch 81/100
Epoch 00081: val_loss did not improve
 - 5s - loss: 0.1900 - acc: 0.9270 - val_loss: 0.1725 - val_acc: 0.9450
Epoch 82/100
Epoch 00082: val_loss did not improve
 - 5s - loss: 0.2018 - acc: 0.9148 - val_loss: 0.1989 - val_acc: 0.9000
Epoch 83/100
Epoch 00083: val_loss did not improve
 - 5s - loss: 0.1895 - acc: 0.9121 - val_loss: 0.2147 - val_acc: 0.8950
Epoch 84/100
Epoch 00084: val_loss did not improve
 - 5s - loss: 0.1859 - acc: 0.9138 - val_loss: 0.1667 - val_acc: 0.9300
Epoch 85/100
Epo

Epoch 40/100
Epoch 00040: val_loss did not improve
 - 5s - loss: 0.2223 - acc: 0.9138 - val_loss: 0.2269 - val_acc: 0.9100
Epoch 41/100
Epoch 00041: val_loss did not improve
 - 5s - loss: 0.2258 - acc: 0.9033 - val_loss: 0.2264 - val_acc: 0.9050
Epoch 42/100
Epoch 00042: val_loss did not improve
 - 5s - loss: 0.2296 - acc: 0.8980 - val_loss: 0.2514 - val_acc: 0.9000
Epoch 43/100
Epoch 00043: val_loss did not improve
 - 5s - loss: 0.2191 - acc: 0.9100 - val_loss: 0.2183 - val_acc: 0.8950
Epoch 44/100
Epoch 00044: val_loss did not improve
 - 5s - loss: 0.2462 - acc: 0.8858 - val_loss: 0.2467 - val_acc: 0.9050
Epoch 45/100
Epoch 00045: val_loss did not improve
 - 5s - loss: 0.2058 - acc: 0.9121 - val_loss: 0.2350 - val_acc: 0.9050
Epoch 46/100
Epoch 00046: val_loss did not improve
 - 5s - loss: 0.2151 - acc: 0.9028 - val_loss: 0.2201 - val_acc: 0.9150
Epoch 47/100
Epoch 00047: val_loss did not improve
 - 5s - loss: 0.2362 - acc: 0.8997 - val_loss: 0.2300 - val_acc: 0.9100
Epoch 48/100
Epo

Epoch 5/100
Epoch 00005: val_loss did not improve
 - 5s - loss: 0.3741 - acc: 0.8345 - val_loss: 0.3268 - val_acc: 0.8200
Epoch 6/100
Epoch 00006: val_loss improved from 0.24083 to 0.21869, saving model to best_m.h5
 - 5s - loss: 0.3937 - acc: 0.8317 - val_loss: 0.2187 - val_acc: 0.9100
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 5s - loss: 0.3664 - acc: 0.8235 - val_loss: 0.2469 - val_acc: 0.8700
Epoch 8/100
Epoch 00008: val_loss did not improve
 - 5s - loss: 0.3759 - acc: 0.8283 - val_loss: 1.5271 - val_acc: 0.5450
Epoch 9/100
Epoch 00009: val_loss did not improve
 - 5s - loss: 0.3496 - acc: 0.8460 - val_loss: 0.2889 - val_acc: 0.8650
Epoch 10/100
Epoch 00010: val_loss did not improve
 - 5s - loss: 0.3869 - acc: 0.8245 - val_loss: 0.2834 - val_acc: 0.8350
Epoch 11/100
Epoch 00011: val_loss did not improve
 - 5s - loss: 0.3409 - acc: 0.8511 - val_loss: 0.2240 - val_acc: 0.8900
Epoch 12/100
Epoch 00012: val_loss did not improve
 - 5s - loss: 0.3021 - acc: 0.8671 - val_loss: 0.

Epoch 68/100
Epoch 00068: val_loss did not improve
 - 5s - loss: 0.2157 - acc: 0.9112 - val_loss: 0.1735 - val_acc: 0.9200
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 5s - loss: 0.2164 - acc: 0.9059 - val_loss: 0.1625 - val_acc: 0.9300
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 5s - loss: 0.1984 - acc: 0.9186 - val_loss: 0.1929 - val_acc: 0.9400
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 5s - loss: 0.2126 - acc: 0.9071 - val_loss: 0.1723 - val_acc: 0.9150
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 5s - loss: 0.2106 - acc: 0.9116 - val_loss: 0.1711 - val_acc: 0.9100
Epoch 73/100
Epoch 00073: val_loss did not improve
 - 5s - loss: 0.2045 - acc: 0.9152 - val_loss: 0.1788 - val_acc: 0.9400
Epoch 74/100
Epoch 00074: val_loss improved from 0.15895 to 0.15705, saving model to best_m.h5
 - 5s - loss: 0.2031 - acc: 0.9236 - val_loss: 0.1570 - val_acc: 0.9250
Epoch 75/100
Epoch 00075: val_loss did not improve
 - 5s - loss: 0.2078 - acc: 0.9203 - val_los

In [12]:
with open('../features/resnet_aug1_fold8_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/resnet_aug1_fold8_sub.csv', index=False)


# pre 2179
# new 2121

0.201390945756
         id  is_iceberg
0  5941774d    0.188701
1  4023181e    0.495063
2  b20200e4    0.058756
3  e7f018bb    0.983047
4  4371c8c3    0.037538
